In [31]:
# importamos las librerias necesarias
import pandas as pd
import json
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np

In [32]:
# Cambiar el directorio de trabajo al directorio donde se encuentran los archivos JSON
%cd ./metadata-sitios
# Obtener una lista de archivos en el directorio actual
archivos = os.listdir()

# Crear un DataFrame vacío para almacenar los datos
df_metadata = pd.DataFrame()

# Iterar sobre cada archivo en la lista de archivos
for archivo in archivos:
    # Verificar si el archivo tiene la extensión .json
    if archivo.endswith('.json'):
        # Leer el archivo JSON y cargarlo en un DataFrame
        data = pd.read_json(archivo, lines=True)
        # Concatenar los datos del archivo al DataFrame principal
        df_metadata = pd.concat([df_metadata, data], axis=0)
print("finalizo cargan de datos")

[WinError 2] El sistema no puede encontrar el archivo especificado: './metadata-sitios'
c:\Users\sebas\OneDrive\Escritorio\Proyecto-Grupal-Google-yelp\Sprint 2 Ingenieria de Datos\ETL_google\metadata-sitios
finalizo cargan de datos


In [33]:
# Eliminamos columnas irrelevantes
df_metadata.drop(columns=['address', 'state', 'relative_results', 'url','hours','description', 'price'], inplace=True)

In [34]:
from pandas import json_normalize 

# Desanidar los datos de la columna "MISC"
df_desanidado = json_normalize(df_metadata['MISC'])

In [35]:
# Vamos a trabajar con una copia de metadata para crear una posible tabla "service_characteristics"
df_gmap_id = df_metadata.copy()
# Resetear el índice del DataFrame df_gmap_id
df_gmap_id.reset_index(drop=True, inplace=True)

# Combinar los DataFrames desanidados
df_service = pd.concat([df_gmap_id, df_desanidado], axis=1)

In [36]:
# Eliminamos columna MISC
df_metadata.drop(columns=['MISC'], inplace=True)

In [37]:
def primer_categoria(lista):
    if lista and len(lista) >= 1:
        return lista[0]
    else:
        return None  

# Aplicar la función a cada fila de la columna 'category'
df_metadata['categoria'] = df_metadata['category'].apply(primer_categoria)

In [38]:
# Eliminamos columna category
df_metadata.drop(columns=['category'], inplace=True)

In [39]:
# Verificar si hay valores duplicados
df_metadata.duplicated().value_counts()

False    2173428
True       26580
Name: count, dtype: int64

In [40]:
df_metadata.drop_duplicates(inplace=True)

In [41]:
# Verificar si hay valores nulos
df_metadata.isnull().sum()

name                 31
gmap_id               0
latitude              0
longitude             0
avg_rating            0
num_of_reviews        0
categoria         13945
dtype: int64

In [43]:
# Eliminar filas con valores nulos en las columnas 'categoria_1' y 'name'
df_metadata = df_metadata.dropna(subset=['categoria', 'name'])

df_metadata.isnull().sum()


name              0
gmap_id           0
latitude          0
longitude         0
avg_rating        0
num_of_reviews    0
categoria         0
dtype: int64

In [44]:
# cambio el nombre de la columna gmap_id a business_id 
df_metadata.rename(columns={'gmap_id': 'business_id'}, inplace=True)

C:\Users\sebas\AppData\Local\Temp\ipykernel_27776\1262992948.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metadata.rename(columns={'gmap_id': 'business_id'}, inplace=True)


In [45]:
df_metadata.head(2)

,name,business_id,latitude,longitude,avg_rating,num_of_reviews,categoria
0,Porter Pharmacy,0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.35710,4.9,16,Pharmacy
1,City Textile,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.21529,4.5,6,Textile exporter


In [46]:
df_metadata.to_csv('metadata.csv', index=False)